## Sales & Profitability Analysis

This notebook analyzes sales and profitability patterns using the cleaned dataset.
The objective is to identify high- and low-performing categories, products, and regions to support data-driven pricing and business decisions.


In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)

df = pd.read_csv("../data/processed/cleaned_sales.csv")
df.head()


,category,city,container,customer_id,customer_name,customer_segment,department,item,item_id,order_date,order_id,order_priority,postal_code,region,row_id,ship_date,ship_mode,state,days_to_ship,discount,kpi_-_profit_ratio,number_of_records,order_quantity,product_base_margin,profit,profit_ratio,sales,shipping_cost,unit_price,profit_margin,profit_per_unit
0,Storage & Organization,Bowie,Large Box,2211,Anita Hahn,Home Office,Office Supplies,Safco Industrial Wire Shelving,10826,2013-01-01,88028,Not Specified,20715,East,19914,2013-01-01,Express Air,Maryland,0,0.08,Poor,1,2,0.00,-425,-2.1907,194,35,96,-2.190722,-212.500000
1,Tables,Phenix City,Jumbo Box,2579,Marshall Sutherland,Home Office,Furniture,Bush Advantage Collection® Round Conference Table,10964,2013-01-01,88296,High,36869,South,23705,2013-01-02,Delivery Truck,Alabama,1,0.09,Poor,1,1,0.64,-274,-1.5657,175,52,213,-1.565714,-274.000000
2,Office Furnishings,Draper,Medium Box,1988,Anna Burgess,Home Office,Furniture,36X48 HARDFLOOR CHAIRMAT,10179,2013-01-01,89999,High,84020,West,19336,2013-01-02,Regular Air,Utah,1,0.05,Poor,1,3,0.78,-181,-2.7424,66,21,21,-2.742424,-60.333333
3,Pens & Art Supplies,Austin,Wrap Bag,117,Linda Weiss,Home Office,Office Supplies,"Dixon Prang® Watercolor Pencils, 10-Color Set ...",11250,2013-01-01,7909,High,78745,Central,1074,2013-01-02,Regular Air,Texas,1,0.03,Poor,1,29,0.44,10,0.0820,122,1,4,0.081967,0.344828
4,Pens & Art Supplies,Lake Oswego,Wrap Bag,114,Ron Newton,Home Office,Office Supplies,"Dixon Prang® Watercolor Pencils, 10-Color Set ...",11250,2013-01-01,89583,High,97035,West,19074,2013-01-02,Regular Air,Oregon,1,0.03,Acceptable,1,7,0.44,19,0.6333,30,1,4,0.633333,2.714286


## Profitability Feature Engineering
Derived additional profitability metrics to evaluate product- and order-level performance beyond raw profit values.

In [3]:
df["profit_margin"] = np.where(
    df["sales"] > 0,
    df["profit"] / df["sales"],
    0
)

df["profit_per_unit"] = np.where(
    df["order_quantity"] > 0,
    df["profit"] / df["order_quantity"],
    0
)

df[["sales", "profit", "profit_margin", "profit_per_unit"]].describe()

,sales,profit,profit_margin,profit_per_unit
count,9426.000000,9426.000000,9426.000000,9426.000000
mean,949.716423,139.233503,-0.194593,-11.310500
std,2598.020092,998.486307,16.584442,376.313847
min,1.000000,-16477.000000,-516.153846,-14141.000000
25%,61.000000,-74.000000,-0.565217,-9.222222
50%,203.000000,3.000000,0.012465,0.233333
75%,776.000000,140.000000,0.516116,13.728571
max,100119.000000,16332.000000,990.000000,4950.000000


### Observations

- Profit margin shows high variance, indicating inconsistent pricing and discount strategies.
- Negative profit and profit-per-unit values confirm the presence of loss-making orders.
- These metrics help identify products and regions contributing to margin erosion.

## Exploratory Profitability Analysis

This section analyzes profitability across categories, regions, and customer segments to identify structural loss drivers and high-performing areas.

### Category-level Profitability Analysis

Aggregated sales and profit metrics at the product category level to identify
which categories drive overall profitability and which consistently generate losses.


In [9]:
category_profit = (
    df.groupby("category")
      .agg(
          total_sales=("sales", "sum"),
          total_profit=("profit", "sum"),
          avg_profit_margin=("profit_margin", "mean"),
          order_count=("order_id", "nunique")
      )
      .sort_values("total_profit", ascending=False)
)


category_profit

,total_sales,total_profit,avg_profit_margin,order_count
category,,,,
Telephones and Communication,1144284,297944,-0.052876,942
Binders and Binder Accessories,638583,226582,-0.800548,976
Office Machines,1218655,168079,-0.121207,373
Chairs & Chairmats,1164586,165341,-0.011216,429
Copiers and Fax,661215,129154,-0.047783,98
Appliances,456736,121641,-0.655688,480
Office Furnishings,444634,92216,-0.262202,851
Computer Peripherals,490851,87911,-0.229802,818
Envelopes,147915,46133,0.126949,270


In [6]:
category_profit[category_profit["total_profit"] < 0]


,total_sales,total_profit,avg_profit_margin,order_count
category,,,,
"Scissors, Rulers and Trimmers",40432,-1938,-0.528398,155
Rubber Bands,8670,-2837,-2.161351,194
Bookcases,507496,-7714,-0.163874,205
Tables,1061922,-72506,-0.320311,398


### Category-level Profitability Analysis

Aggregated sales and profit metrics at the product category level to identify
which categories drive overall profitability and which consistently generate losses.


In [10]:
region_profit = (
    df.groupby("region")
      .agg(
          total_sales=("sales", "sum"),
          total_profit=("profit", "sum"),
          avg_profit_margin=("profit_margin", "mean"),
          order_count=("order_id", "nunique")
      )
      .sort_values("total_profit", ascending=False)
)

region_profit


,total_sales,total_profit,avg_profit_margin,order_count
region,,,,
East,3143122,475024,-0.291902,2172
Central,2294603,458668,-0.310205,1796
West,2002353,271578,-0.470128,1389
South,1511949,107145,0.441460,1267


In [12]:
region_profit[region_profit["total_profit"] < 0]


,total_sales,total_profit,avg_profit_margin,order_count
region,,,,


#### Observations

- All regions are profitable in absolute terms, indicating that losses are not region-wide.
- East and Central regions generate the highest profits primarily due to high sales volume,
  despite operating at negative average profit margins.
- The South region stands out as the only region with a positive average profit margin,
  suggesting more efficient pricing or lower cost structures.
- This indicates margin inefficiencies rather than demand issues in most regions.

## Customer Segment Profitability Analysis

This section evaluates profitability across customer segments to identify
which segments contribute most to profit and which may require pricing,
discount, or retention strategy adjustments.


In [13]:
segment_profit = (
    df.groupby("customer_segment")
      .agg(
          total_sales=("sales", "sum"),
          total_profit=("profit", "sum"),
          avg_profit_margin=("profit_margin", "mean"),
          order_count=("order_id", "nunique")
      )
      .sort_values("total_profit", ascending=False)
)

segment_profit


,total_sales,total_profit,avg_profit_margin,order_count
customer_segment,,,,
Corporate,3269431,505515,0.000095,2298
Small Business,1678389,316470,-0.755011,1264
Home Office,2168965,283884,0.105607,1598
Consumer,1835242,206546,-0.363868,1300


In [14]:
segment_profit[segment_profit["total_profit"] < 0]


,total_sales,total_profit,avg_profit_margin,order_count
customer_segment,,,,


### Customer Segment Profitability Insights

- All customer segments are profitable in aggregate, indicating no segment-wide loss issues.
- Corporate customers generate the highest total profit but operate at near-zero average margins, suggesting aggressive pricing or heavy discounting.
- Home Office is the most margin-efficient segment despite lower sales volume, indicating healthier pricing dynamics.
- Small Business and Consumer segments show negative average margins, implying profitability is driven by volume rather than per-order efficiency.

These findings suggest margin optimization opportunities through pricing and discount strategy refinement rather than customer segment exclusion.


## State-level Profitability Analysis

This section analyzes profitability across states to identify geographic pockets
of margin erosion and high-performing markets.


In [15]:
state_profit = (
    df.groupby("state")
      .agg(
          total_sales=("sales", "sum"),
          total_profit=("profit", "sum"),
          avg_profit_margin=("profit_margin", "mean"),
          order_count=("order_id", "nunique")
      )
      .sort_values("total_profit", ascending=False)
)

state_profit


,total_sales,total_profit,avg_profit_margin,order_count
state,,,,
Texas,998144,172406,-0.343166,714
New York,532726,112053,-0.355256,331
Massachusetts,1090616,93819,-0.226190,658
California,975048,80333,-0.334460,576
Connecticut,229756,64073,-0.253109,291
Michigan,324590,53044,-0.338453,229
Oregon,196863,51429,-1.407392,190
Ohio,222359,48535,-0.357244,219
Wisconsin,148725,40782,-0.345992,111


In [16]:
state_profit[state_profit["total_profit"] < 0]


,total_sales,total_profit,avg_profit_margin,order_count
state,,,,
Tennessee,94886,-6904,-1.860634,112
Mississippi,41921,-18047,1.509692,60


### State-level Profitability Insights

- Most states are profitable in absolute terms despite negative average margins, indicating volume-driven profitability.
- Tennessee and Mississippi are the only states with net losses, making them priority candidates for corrective action.
- Negative margins in high-revenue states suggest pricing, discounting, or logistics inefficiencies rather than weak demand.
- State-level analysis helps isolate geographically concentrated loss drivers for targeted intervention.


## Key Business Takeaways

- Profitability issues are driven more by margin inefficiencies than lack of demand.
- Certain categories and products consistently generate losses despite strong sales volumes.
- The South region stands out with positive margins, indicating better pricing or cost control.
- Losses are concentrated in specific states rather than across entire regions.
- Targeted pricing optimization, discount control, and cost rationalization can significantly improve overall profitability.